W dwóch notebookach postaramy się stworzyć system który będzie kontrolował prostego robota.
Do przeprowadzenia badania potrzebujemy środowiska symulacyjnego, dlatego musimy skorzystać z biblioteki openai gym.

In [3]:
import gym
import time
import numpy as np

Symulacja Cartpole to prosta kontrola wózka do którego doczepiony jest maszt. Wózek kontrolowany jest poprzez aplikowanie wartości 1 lub -1. Nie możemy dopuścić do tego, żeby maszt się przewrócił na ziemię. Jesteśmy nagradzani 1 punktem za każdy moment w czasie (z ang. timestep) w którym maszt się prosto w górze. Za każdym razem gdy maszt odchyli się o więcej niż 15 stopni następuje reset.

Korzystamy z metody make do wczytania modułu Cartpole w wersji 0.

In [4]:
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)

Metoda reset na początku sprawia, że za każdym razem wczytujemy środowisko w nienaruszonym stanie, możemy potraktować to jako wstępne wczytanie.
Pętla for ustawia symulacje na 1000 momentów w czasie.
Metodzie render zawdzięczamy to, że python wyświetla nam symulacje w postaci graficznej.
Metoda step to po prostu akcja wykonywana w momencie czasu. Wczytujemy do niej losowe akcje, bo na tym etapie wynik nas nie interesuje.

In [5]:
env.reset()
for i in range(1000):
    env.render()
    env.step(env.action_space.sample())
    
# Tutaj napotkałem lekki problem z działaniem programu i musiałem skorzystać ze sztucznego spowolnienia działania
# programu w każdej iteracji pętli.
    time.sleep(0.02)
    
    if i%10==0: print(i)
#    print(obs)
env.close()

0
10


C:\Users\Patryk\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [6]:
np.random.seed(1)
env.seed(1)

[1]

Nasze wartości dzięki którym możemy porównać jak poszczególne iteracje algorytmu sobie radzą. Naszym celem jest ich maksymalizacja.

In [7]:
hi_rew = 0
best_weights = None

Tym razem w pętli mamy 200 powtórzeń zamiast tysiąca. Reset na początku sprawia, że wracamy do początkowych wartości za każdą iteracją. Dla każdej iteracji przyjmujemy unikatowe wagi dla 4 wartości które możemy zaobserwować w środowisku, czyli pozycja i prędkość wózka oraz kąt pochylenia masztu i prędkość masztu, liczby weźmiemy sobie z przedziału od -1 do 1. Określamy również zmienną sum_rew w której będziemy sobie dodawać wartości nagrody z każdego momentu w czasie. Wewnętrzna pętla będzie miała 1000 iteracji i powinna określić max nagrodę którą otrzymamy przy użyciu losowych wag określonych wcześniej. Akcja to ruch wózka któy będzie równy 0, czyli będzie miał ruszyć się w lewo, jeśli maszt będzie pochylony w lewo. Wtedy nasz wózek będzie musiał przyspieszyc w te samą stronę, żeby osiągnąć balans. W przeciwnym wypadku wózek będzie jechać w prawo. Później przywołujemy wszystkie wartości ze środowiska, w przeciwieństwie do operacji piątej w notebooku gdzie nasza akcja była całkowicie losowa. To tutaj akcja będzie podjęta w kolejnej iteracji na podstawie obserwacji z obecnej. Po czym aktualizujemy zmienną sum_rew o wynik. Po czym sprawdzamy wynik i aktualizujemy go jeśli obecna iteracja jest najlepsza.

In [8]:
for i in range(200):
    obs = env.reset()
    weights = np.random.uniform(-1, 1, 4)
    sum_rew = 0
    for j in range(1000):
        env.render()
        action = 0 if np.matmul(weights, obs) < 0 else 1
        obs, rew, done, info = env.step(action)
        sum_rew += rew
        print(i, j, weights, obs, action, sum_rew, best_weights)
    if sum_rew > hi_rew:
        hi_rew = sum_rew
        best_weights = weights
    time.sleep(0.02)
env.close()

0 0 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.03076804  0.19700098 -0.03151444 -0.33357875] 1 1.0 None
0 1 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.03470806  0.39255697 -0.03818601 -0.63603079] 1 2.0 None
0 2 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.0425592   0.5881901  -0.05090663 -0.94049052] 1 3.0 None
0 3 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.054323    0.78395987 -0.06971644 -1.24872484] 1 4.0 None
0 4 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.0700022   0.97990287 -0.09469093 -1.5624054 ] 1 5.0 None
0 5 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.08960026  1.17602127 -0.12593904 -1.88306267] 1 6.0 None
0 6 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.11312068  1.37226925 -0.16360029 -2.21203155] 1 7.0 None
0 7 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.14056607  1.56853737 -0.20784093 -2.55038687] 1 8.0 None
0 8 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 0.17193682  1.76463456 -

0 93 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [15.88414129 17.14151706 -4.42376554  4.67777914] 1 9.0 None
0 94 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [16.22697163 17.34718728 -4.33020996  5.04742906] 1 9.0 None
0 95 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [16.57391537 17.55711007 -4.22926138  5.4376521 ] 1 9.0 None
0 96 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [16.92505757 17.77138217 -4.12050834  5.84731322] 1 9.0 None
0 97 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [17.28048522 17.98981932 -4.00356207  6.27410831] 1 9.0 None
0 98 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [17.6402816  18.21182083 -3.87807991  6.71405708] 1 9.0 None
0 99 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [18.00451802 18.43621188 -3.74379877  7.16088816] 1 9.0 None
0 100 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [18.37324226 18.66108753 -3.600581    7.60540393] 1 9.0 None
0 101 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [18.74646401 18

0 183 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [58.34871177 22.88544185  4.45611665 -2.04505832] 0 9.0 None
0 184 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [58.80642061 22.69583844  4.41521548 -2.40154672] 0 9.0 None
0 185 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [59.26033738 22.5041447   4.36718455 -2.76685739] 0 9.0 None
0 186 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [59.71042027 22.31000651  4.3118474  -3.1420546 ] 0 9.0 None
0 187 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [60.1566204  22.11308411  4.24900631 -3.52795961] 0 9.0 None
0 188 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [60.59888208 21.91307632  4.17844712 -3.92505423] 0 9.0 None
0 189 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [61.03714361 21.70975703  4.09994603 -4.33334475] 0 9.0 None
0 190 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [61.47133875 21.50302659  4.01327914 -4.7521732 ] 0 9.0 None
0 191 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [61.9013

0 273 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [84.99892776  6.63369029  4.3879918   5.256941  ] 0 9.0 None
0 274 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.13160156  6.42255954  4.49313062  4.87733223] 0 9.0 None
0 275 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.26005275  6.21613037  4.59067727  4.52302157] 0 9.0 None
0 276 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.38437536  6.01403823  4.6811377   4.19439206] 0 9.0 None
0 277 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.50465612  5.81580374  4.76502554  3.8912445 ] 0 9.0 None
0 278 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.6209722   5.6209047   4.84285043  3.6130328 ] 0 9.0 None
0 279 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.73339029  5.42882216  4.91511109  3.35901374] 0 9.0 None
0 280 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.84196674  5.23906782  4.98229136  3.12834092] 0 9.0 None
0 281 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [85.9467

0 364 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 81.95419143 -10.26082649  13.08234859   6.4057682 ] 0 9.0 None
0 365 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 81.7489749  -10.43917465  13.21046395   6.7835173 ] 0 9.0 None
0 366 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 81.5401914  -10.6097257   13.3461343    7.16462688] 0 9.0 None
0 367 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 81.32799689 -10.77097336  13.48942684   7.54333154] 0 9.0 None
0 368 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 81.11257742 -10.92171233  13.64029347   7.9142123 ] 0 9.0 None
0 369 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 80.89414318 -11.0612367   13.79857771   8.27242359] 0 9.0 None
0 370 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 80.67291844 -11.18952705  13.96402619   8.61365026] 0 9.0 None
0 371 [-0.16595599  0.44064899 -0.99977125 -0.39533485] [ 80.4491279  -11.3074101   14.13629919   8.93371734] 0 9.0 None
0 372 [-0.16595599  0.44064899 -

...

8 779 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1104.94222987  -142.42099937  -122.53221293   -13.81967018] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 780 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1107.79064986  -142.6140933   -122.80860633   -14.10632713] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 781 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1110.64293172  -142.74967487  -123.09073287   -14.21832881] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 782 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1113.49792522  -142.83037509  -123.37509945   -14.16202134] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 783 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1116.35453272  -142.86938346  -123.65833987   -13.97905239] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 784 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1119.21192039  -142.88515208  -123.93792092   -13.7223377 ] 0 9.0 [-0.20

8 848 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1309.12198985  -154.35622215  -139.9580032    -11.68714712] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 849 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1312.2091143   -154.66326084  -140.19174615   -12.04959528] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 850 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1315.30237951  -154.97490895  -140.43273805   -12.49952578] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 851 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1318.40187769  -155.28508609  -140.68272857   -13.01157193] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 852 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1321.50757941  -155.58342642  -140.94296      -13.5439158 ] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 853 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1324.61924794  -155.85493227  -141.21383832   -14.03653389] 0 9.0 [-0.20

8 915 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1524.00669121  -166.01053455  -157.62799093   -12.01830618] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 916 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1527.3269019   -166.26762287  -157.86835706   -11.8424729 ] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 917 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1530.65225436  -166.54252027  -158.10520651   -11.76045135] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 918 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1533.98310477  -166.83121147  -158.34041554   -11.78724836] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 919 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1537.31972899  -167.13133881  -158.57616051   -11.92989279] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 920 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1540.66235577  -167.44131179  -158.81475836   -12.18858224] 0 9.0 [-0.20

8 983 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1758.7668583   -178.27300066  -176.03788382   -12.9772501 ] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 984 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1762.33231831  -178.48421778  -176.29742883   -12.69418768] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 985 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1765.90200267  -178.72879657  -176.55131258   -12.45774592] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 986 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1769.4765786   -178.99865696  -176.8004675    -12.30012806] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 987 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1773.05655174  -179.28728811  -177.04647006   -12.24633093] 0 9.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]
8 988 [ 0.91577906  0.06633057  0.38375423 -0.36896874] [-1776.6422975   -179.59036646  -177.29139668   -12.31143068] 0 9.0 [-0.20

KeyboardInterrupt: 

In [9]:
print(hi_rew, best_weights)

85.0 [-0.20646505  0.07763347 -0.16161097  0.370439  ]


Tutaj testujemy nasze najlepsze wagi sprawdzając czy są one w stanie utrzymać maszt w pionie przez 100 iteracji. Co oczywiście się udaje. Byliśmy w stanie osiągnąć cel, ale nadal możemy poprawić wynik stosując deep learning.

In [10]:
obs = env.reset()
for j in range(100):
    env.render()
    action = 0 if np.matmul(weights, obs) < 0 else 1
    obs, rew, done, info = env.step(action)
    print(j, action)
env.close()

0 0
1 1
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
